<div align="center">
    <a href="https://github.com/syubogdanov/hse-howto-python">
        <img src="https://cdn-icons-png.flaticon.com/128/1864/1864652.png" height="128px" width="auto">
    </a>
    <h3>
        <b>
            Продвинутый Python
        </b>
    </h3>
    <i>
        Взаимодействие с ОС. Консольные утилиты
    </i>
</div>

<br>

**Цель занятия.** Изучение дополнительных инструментов, полезных при разработке программного обеспечения. В частности: углубление в аннотацию типов, взаимодействие с операционной системой и работа с аргументами командной строки.

**Примечание.** Начало занятия будет посвящено изучению дополнительного функционала модуля `typing`.

**Пример.** Напишите функцию, которая будет вызывать произвольное исключение.

In [34]:
from typing import NoReturn
from random import choice

EXCEPTIONS: list[Exception] = [
    RuntimeError,
    TypeError,
    ValueError,
]


def anyraise() -> NoReturn:
    exception = choice(EXCEPTIONS)
    raise exception("[!] Random Exception")

**Пояснение.** У невозвратных функций выходной тип данных принято параметризовать при помощи `typing.NoReturn`.

**Пример.** Напишите функцию, которая имитирует русскую рулетку при барабане револьвера в `n` патронов. В случае поражения бросайте произвольное исключение при помощи функции из примера выше.

In [35]:
from random import randint
from typing import assert_never, Never, Optional


def randbool() -> bool:
    return bool(randint(0, 1))


def roulette(n: int) -> Optional[NoReturn]:
    for _ in range(n):
        if randbool():
            anyraise()
            assert_never(Never)

In [60]:
roulette(3)

**Замечание.** Вы можете помечать недостижимые участки кода при помощи `assert_never(Never)`. Его исполнение приведет к вызову исключения. Как правило, объект используется сразу же после вызова функции, имеющей атрибут `typing.NoReturn`.

**Пример.** Напишите функцию, которая конкатенирует две строки (возможно, переданные в виде байтов).

In [6]:
from typing import AnyStr

def concatenate(lh: AnyStr, rh: AnyStr) -> AnyStr:
    return lh + rh

In [9]:
concatenate("left", "right")

'leftright'

In [10]:
concatenate(b"left", b"right")

b'leftright'

In [11]:
concatenate("left", b"right")

TypeError: can only concatenate str (not "bytes") to str

**Пояснение.** Параметризация аргументов функции при помощи `typing.AnyStr` эквивалентна параметризации `str | bytes`. В то же время использование `typing.Anystr` накладывает ограничение на смешение типов - все аргументы, отмеченные при помощи `typing.Anystr`, в рамках одного вызова должны строго относиться либо к `str`, либо к `bytes`. Правило касается и выходного параметра.

**Пример.** Напишите функцию, которая принимает на вход одиночный символ, и возвращает строку, состоящую из всех букв алфавита того же регистра, начиная с переданного символа.

In [17]:
from typing import TypeAlias, TypeVar

char: TypeAlias = TypeVar("char", bound=str)


def endalph(letter: char) -> str:
    items: list[char] = []

    for shift in range(ord('z') - ord(letter.lower()) + 1):
        items += chr(ord(letter) + shift)

    return "".join(items)

In [18]:
endalph('a')

'abcdefghijklmnopqrstuvwxyz'

In [19]:
endalph('T')

'TUVWXYZ'

**Пояснение.** Объект `typing.TypeVar` используется для создания аннотации для произвольного типа данных. В Python одиночные символы имеют тип данных `str`, однако для задачи допустимо уточнить входной тип данных через новый объект аннотации `char`, являющегося подтипом `str`.

**Примечание.** Использование `str` в аннотации тоже было бы правильным. Однако для внешнего пользователя в документации стоило бы указать, какой входной аргумент ожидается.

**Замечание.** Если необходимо указать спецификацию под несколько допустимых типов данных - например, для `str` и `int`, тогда можно использовать любой из:

- `T: TypeAlias = TypeVar("T", bound=Union[str, int])`
- `T: TypeAlias = TypeVar("T", str, int)`

**Замечание.** Если Вы указываете единственный тип в аргументах `typing.TypeVar`, тогда не забудьте указать, что он относится к параметру `bound`. В противном случае - получите исключение.

**Пример.** Напишите функцию, которая проверяет, что строка относится к типу `char`.

In [24]:
from typing import TypeGuard


def is_char(value: AnyStr) -> TypeGuard[char]:
    if not isinstance(value, str):
        return False

    return len(value) == 1

In [25]:
is_char("abc")

False

In [26]:
is_char('a')

True

In [27]:
is_char(b'a')

False

**Пояснение.** При написании кода Вы периодически используете функции-валидаторы. Для них принято использовать объект аннотации `typing.TypeGuard`. Он показывает, что функция возвращает `True` тогда и только тогда, когда ее аргумент относится к типу данных, указанному в `typing.TypeGuard`. Для лучшего понимания стоит еще раз изучить пример выше.

**Замечание.** Подразумевается, что входной тип данных является надмножеством того, что указано в `typing.TypeGuard`.

**Пример.** Покажите, что переменная не предполагает изменения значения.

In [20]:
from typing import Final

MEMORY_LIMIT: Final[int] = 1024 * 1024  # KiB

**Пояснение.** Объект `typing.Final` показывает, что некоторое значение является конечным и не может быть изменено в теле программы. Правило распространяется на переменную даже при наследовании. Попытка переприсвоить значение не вызовет исключений, хотя Ваш линтер, скорее всего, возмутится, заметив подобные действия.

**Пример.** Покажите, что метод класса не может быть переопределен при наследовании.

In [ ]:
from typing import final

class Example(object):
    @final
    def example() -> None:
        ...

**Пример.** Покажите, что класс не может иметь дочерние классы.

In [ ]:
@final
class Example(object):
    ...

**Пояснение.** Декоратор `typing.final` показывает разработчику, что метод класса не может быть переопределен в наследнике или что класс не может иметь наследников. В случае нарушения соглашения никаких исключений вызвано не будет, хотя Ваш линтер, скорее всего, возмутится, заметив подобные действия.

**Пример.** Получить аннотацию функции, не используя магические атрибуты.

In [28]:
def function(a: int, b: str) -> list[float]:
    return list(len(b) / a for _ in range(a))

In [29]:
from typing import get_type_hints

get_type_hints(function)

{'a': int, 'b': str, 'return': list[float]}

**Примечание.** Модуль `platform` предназначен для доступа к идентифицирующим данным Вашей платформы.

**Пример.** Узнать, на какой операционной системе запускается программа.

In [61]:
import platform

platform.system()

'Windows'

**Пример.** Узнать, на каком процессоре работает компьютер.

In [62]:
platform.processor()

'Intel64 Family 6 Model 158 Stepping 10, GenuineIntel'

**Пример.** Узнать сетевое имя компьютера.

In [63]:
platform.node()

'DESKTOP-U10TTDV'

**Пример.** Узнать архитектуру компьютера.

In [64]:
platform.architecture()

('64bit', 'WindowsPE')

**Примечание.** Модуль `platform` позволяет также узнать информацию о текущем Python на устройстве.

**Пример.** Узнать информацию о Python на данном устройстве.

In [65]:
print(f"Компилятор: {platform.python_compiler()}")
print(f"Реализация: {platform.python_implementation()}")
print(f"Версия:     {platform.python_version()}")

Компилятор: MSC v.1933 64 bit (AMD64)
Реализация: CPython
Версия:     3.11.0


**Замечание.** В зависимости от платформы модуль `platform` может предложить дополнительные функции. В частности, специальный инструментарий определен для `Java Platform`, `Windows Platform`, `macOS Platform`, `Unix Platforms` и `Linux Platforms`.

**Пример.** Получить информации о `Windows` на данном компьютере.

In [66]:
print(f"Версия:  {platform.win32_ver()}")
print(f"Издание: {platform.win32_edition()}")

Версия:  ('10', '10.0.19045', 'SP0', 'Multiprocessor Free')
Издание: Core


**Пример.** Получить информацию о `macOS` на данном компьютере.

In [67]:
print(f"Версия: {platform.mac_ver()}")

Версия: ('', ('', '', ''), '')


**Замечание.** При невозможности получения каких-либо данных функции модуля `platform` вернут пустую строку. Правило касается как обычных функций, так и платформозависимых. В примере выше невозможность получения данных обусловлена тем, что платформа не относится к `macOS`.

**Примечание.** Модуль `os` предназначен для прямого взаимодействия с операционной системой и ее компонентами.

**Пример.** Узнать, в какой директории находится пользователь в настоящий момент времени.

In [68]:
import os

os.getcwd()

'c:\\Users\\FCOla\\Documents\\GitHub\\hse-howto-python\\lessons\\04'

**Пример.** Узнать содержимое директории, в которой находится пользователь.

In [72]:
os.listdir()

['homework.ipynb', 'lecture.ipynb', 'seminar.ipynb']

In [70]:
os.listdir(r"./..")

['01', '02', '03', '04', '05', '06']

**Замечание.** Если аргумент-путь отсутствует, то функция `os.listdir` выведет содержимое текущей директории.

**Пояснение.** При работе с директориями обычно используют `r`-строки. Они отличаются от обычных только тем, что отключают экранирование. Например, `\n` будет распознано не как перевод строки, а как два символа - `\` и `n`.

**Пример.** Создать в директории, где находится пользователь, пустую папку.

In [73]:
os.mkdir(r"./temp")

In [74]:
os.listdir()

['homework.ipynb', 'lecture.ipynb', 'seminar.ipynb', 'temp']

**Пример.** Переименовать созданную в примере выше директорию.

In [75]:
os.rename(r"./temp", r"./abc")

In [76]:
os.listdir()

['abc', 'homework.ipynb', 'lecture.ipynb', 'seminar.ipynb']

**Пример.** Удалить директорию, созданную в примере выше.

In [77]:
os.rmdir(r"./abc")

In [78]:
os.listdir()

['homework.ipynb', 'lecture.ipynb', 'seminar.ipynb']

**Пример.** Удалить файл.

In [81]:
os.remove(r"./homework.ipynb")

In [83]:
os.listdir()

['lecture.ipynb', 'seminar.ipynb']

**Замечание.** Функция `os.remove` предназначена только для работы с файлами. Использовать ее с папками не получится - получите исключение.

**Пример.** Итерирование по директории с возможностью получения подробной информации об объектах.

In [4]:
from collections.abc import Iterator


iterator: Iterator[os.DirEntry] = os.scandir(r"./../..")

for file in iterator:
    print(file)

<DirEntry '.git'>
<DirEntry '.gitignore'>
<DirEntry '.markdownlint.json'>
<DirEntry 'lessons'>
<DirEntry 'LICENSE'>
<DirEntry 'projects'>
<DirEntry 'README.md'>
<DirEntry 'solutions'>


**Пояснение.** Получаемые при помощи итератора объекты относятся к сущностям типа `os.DirEntry`. Скорее всего, наиболее часто Вы будете использовать следующие ее атрибуты и методы:

- `name` - имя файла без учета префикса;
- `path` - директория, в которой содержится файл;
- `is_dir()` - является ли объект папкой;
- `is_file()` - является ли объект файлом;
- `is_symlink()` - является ли объект символической ссылкой.

**Замечание.** Методы `is_dir` и `is_file` также вернут `True`, если сущность является символической ссылкой (ярлыком) на требуемый тип файла. Чтобы проверять строгую принадлежность, используйте вызов метода с аргументом `follow_symlinks`, принимающим значение `False`.

**Пример.** Итерирование по директории с возможностью получения подробной информации об объектах.

In [5]:
def to_filetype(entity: os.DirEntry) -> str:
    if entity.is_dir(follow_symlinks=False):
        return "Папка"

    if entity.is_file(follow_symlinks=False):
        return "Файл"

    if entity.is_symlink():
        return "Ярлык"

    assert_never(Never)


for entity in os.scandir(r"./../.."):
    filetype: str = to_filetype(entity)

    print(f"Имя: {entity.name}")
    print(f"Тип: {filetype}")

    print()

Имя: .git
Тип: Папка

Имя: .gitignore
Тип: Файл

Имя: .markdownlint.json
Тип: Файл

Имя: lessons
Тип: Папка

Имя: LICENSE
Тип: Файл

Имя: projects
Тип: Папка

Имя: README.md
Тип: Файл

Имя: solutions
Тип: Папка



**Замечание.** Документация к функции `os.scandir` просит закрывать полученный итератор по окончании работы с ним.

**Пример.** Итерирование по директории с возможностью получения подробной информации об объектах.

In [ ]:
iterator: Iterator[os.DirEntry] = os.scandir(r"./../..")

for entity in iterator:
    filetype: str = to_filetype(entity)
    print(f"Имя: {entity.name}")
    print(f"Тип: {filetype}")

iterator.close()

**Пример.** Итерирование по директории с возможностью получения подробной информации об объектах.

In [ ]:
with os.scandir(dir) as iterator:
    for file in iterator:
        filetype: str = to_filetype(entity)
        print(f"Имя: {entity.name}")
        print(f"Тип: {filetype}")

**Замечание.** Использование менеджера контекста является более предпочтительным по причине наличия гарантии на закрытие итератора.

**Пример.** Эффективное итерирование по всем папкам, которые встречаются в директории.

In [10]:
for root, dirs, files in os.walk(r"./../"):

    print(f"Изучаемая директория: {root}")
    print(f"Вложенные папки:      {dirs}")
    print(f"Файлы в директории:   {files}")

    print()

Изучаемая директория: ./../
Вложенные папки:      ['01', '02', '03', '04', '05', '06']
Файлы в директории:   []

Изучаемая директория: ./../01
Вложенные папки:      []
Файлы в директории:   ['homework.ipynb', 'lecture.ipynb', 'seminar.ipynb']

Изучаемая директория: ./../02
Вложенные папки:      []
Файлы в директории:   ['homework.ipynb', 'lecture.ipynb', 'seminar.ipynb']

Изучаемая директория: ./../03
Вложенные папки:      []
Файлы в директории:   ['homework.ipynb', 'lecture.ipynb', 'seminar.ipynb']

Изучаемая директория: ./../04
Вложенные папки:      []
Файлы в директории:   ['homework.ipynb', 'lecture.ipynb', 'seminar.ipynb']

Изучаемая директория: ./../05
Вложенные папки:      []
Файлы в директории:   ['homework.ipynb', 'lecture.ipynb', 'seminar.ipynb']

Изучаемая директория: ./../06
Вложенные папки:      []
Файлы в директории:   ['homework.ipynb', 'lecture.ipynb', 'seminar.ipynb']



**Замечание.** В отличие от `os.scandir`, функция `os.walk` не требует, чтобы итератор был закрыт по окончании работы.

**Замечание.** Функция `os.walk` поддерживает возможность перехода по символическим ссылкам. В таком случае итератор будет дополнительно перебирать директории, которые доступны при помощи подобных переходов. Обратите внимание, что существует вероятность попасть в бесконечный цикл.

**Примечание.** Модуль `os.path` предназначен для упрощения работы с директориями и файлами.

**Пример.** Объединить узлы в единый путь.

In [13]:
import os.path

lh: str = "C:\\"
mid: str = r"hse-howto-python\lessons"
rh: str = r"04\lecture.ipynb"

os.path.join(lh, mid, rh)

'C:\\hse-howto-python\\lessons\\04\\lecture.ipynb'

**Пояснение.** Функция `os.path.join` принимает неограниченное число аргументов и объединяет их в единый путь. Обратите внимание, что используемые разделители узлов - платформозависимые. Например, на `Windows` разделителем выступает `\`, а на `Linux` - `/`.

**Пример.** Получить абсолютный путь до файла.

In [14]:
filepath: str = r"./lecture.ipynb"
os.path.abspath(filepath)

'c:\\Users\\FCOla\\Documents\\GitHub\\hse-howto-python\\lessons\\04\\lecture.ipynb'

**Пример.** Проверка сущности на принадлежность типу.

In [15]:
filepath: str = r"./lecture.ipynb"
os.path.isdir(filepath)

False

In [16]:
filepath: str = r"./lecture.ipynb"
os.path.islink(filepath)

False

In [17]:
filepath: str = r"./lecture.ipynb"
os.path.isfile(filepath)

True

**Примечание.** Модуль `sys` предназначен для работы с системными функциями и параметрами.

**Пример.** Узнать максимальное возможное число на данном устройстве.

In [18]:
import sys

sys.maxsize

9223372036854775807

**Примечание.** `9223372036854775807` - это то же, что и `2 ** 63 - 1`

**Пример.** Узнать тип представления чисел на данном устройстве.

In [19]:
sys.byteorder

'little'

**Пример.** Запретить создание файлов `__pycache__`.

In [21]:
sys.dont_write_bytecode: bool = True

# Body of Your Program

**Пример.** Завершить программу с произвольным кодом возврата.

In [ ]:
code: int = 0
sys.exit(code)

**Пример.** Получить аргументы командной строки.

In [ ]:
sys.argv

**Рассуждение.** Использование `sys.argv` позволяет узнать, какие аргументы отправил внешний пользователь Вашей программе. Тем не менее, задача их обработки лежит строго на разработчике. Стоит отметить, что эта задача достаточно трудоемка: нужно проверить, что поступившие данные корректны, не противоречат друг другу и так далее. Автоматизировать этот процесс позволяет использование модуля `argparse`. 

**Примечание.** Модуль `argparse` предназначен для анализа и обработки параметров командной строки, аргументов и вложенных команд.

**Замечание.** Ввиду того, что `argparse` не предназначен для работы с `.ipynb` файлами, будем предполагать, что код ниже относится к некоторому проекту со своей иерархией. Мы будем указывать, в каком файле что находится.

**Пример.** Создание пустого обработчика аргументов командной строки.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser()
parser.parse_args()

**Запуск.** Запустите в терминале следующую команду: `python ./main.py -h`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
usage: main.py [-h]

options:
  -h, --help  show this help message and exit
```

**Пояснение.** Использование модуля `argparse` автоматически выстраивает консольный интерфейс для Вашей программы.

**Пример.** Добавление информации в интерфейс консольного приложения.

In [ ]:
# %file: ./main.py

import argparse
import sys

parser = argparse.ArgumentParser(
    prog=sys.argv[0],
    description="The description of the project",
    epilog="by @maseoff",
)

parser.parse_args()

**Запуск.** Запустите в терминале следующую команду: `python ./main.py -h`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
usage: ./main.py [-h]

The description of the project

options:
  -h, --help  show this help message and exit

by @maseoff
```

**Пояснение.** Внутри обработчика можно задать определенные настройки. Например:

- `prog` - имя программы, которое будет показано в поле `usage`;
- `description` - описание программы, которое будет сразу после поля `usage`;
- `epilog` - эпилог для программы.

**Примечание.** По умолчанию атрибут `prog` содержит только название конечного файла, чего вполне достаточно.

**Примечание.** В эпилоге, как правило, указывают автора программы.

**Пример.** Добавление позиционного аргумента в интерфейс консольного приложения.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument("arg")

parser.parse_args()

**Запуск.** Запустите в терминале следующую команду: `python ./main.py -h`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
usage: main.py [-h] arg

The description of the project

positional arguments:
  arg

options:
  -h, --help  show this help message and exit

by @maseoff
```

**Пример.** Взаимодействие с позиционным аргументом.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument("arg")

args = parser.parse_args()

print("arg:", args.arg)

**Запуск.** Запустите в терминале следующую команду: `python ./main.py 42`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
arg: 42
```

**Пояснение.** Метод `parse_args` возвращает объект, в атрибутах которого хранятся параметры командной строки. Обратившись по имени аргумента, Вы получаете его значение.

**Замечание.** По умолчанию аргументы командной строки интерпретируются как объекты типа `str`, поэтому в примере выше `args.arg` возвращает строку `"42"`.

**Пример.** Задать тип данных аргументу.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument("arg", type=int)

args = parser.parse_args()

print("arg:", args.arg)
print("type:", type(args.arg))

**Запуск.** Запустите в терминале следующую команду: `python ./main.py 42`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
arg: 42
type: <class 'int'>
```

**Упражнение.** Убедитесь в том, что по умолчанию возвращается строковый тип данных.

**Замечание.** Параметр `type` указывает на класс, от которого должно быть проинициализировано значение. Обратите внимание, что инициализация будет произведена от аргумента, являющегося строкой.

**Упражнение.** Используя реализацию класса `uint32_t` из предыдущих занятий, добавьте в консольную утилиту возможность получать беззнаковые 32-битные числа.

**Пример.** Добавление позиционных аргументов.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument("arg1", type=int)
parser.add_argument("arg2", type=str)
parser.add_argument("arg3", type=float)

args = parser.parse_args()

print("arg1:", args.arg1)
print("type:", type(args.arg1))

print("arg2:", args.arg2)
print("type:", type(args.arg2))

print("arg3:", args.arg3)
print("type:", type(args.arg3))

**Запуск.** Запустите в терминале следующую команду: `python ./main.py -h`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
usage: main.py [-h] arg1 arg2 arg3

The description of the project

positional arguments:
  arg1
  arg2
  arg3

options:
  -h, --help  show this help message and exit

by @maseoff
```

**Запуск.** Запустите в терминале следующую команду: `python ./main.py -h`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
arg1: 42
type: <class 'int'>
arg2: forty-two
type: <class 'str'>
arg3: 42.0
type: <class 'float'>
```

**Пример.** Добавление подсказки к аргументам.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument("arg1", type=int, help="The first argument")
parser.add_argument("arg2", type=str, help="The second argument")
parser.add_argument("arg3", type=float, help="The third argument")

parser.parse_args()

**Запуск.** Запустите в терминале следующую команду: `python ./main.py -h`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
usage: main.py [-h] arg1 arg2 arg3

The description of the project

positional arguments:
  arg1        The first argument
  arg2        The second argument
  arg3        The third argument

options:
  -h, --help  show this help message and exit

by @maseoff
```

**Пример.** Добавление опционального аргумента в интерфейс консольного приложения.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument("required", help="The required argument")
parser.add_argument("--optional", help="The optional argument")

parser.parse_args()

**Запуск.** Запустите в терминале следующую команду: `python ./main.py -h`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
usage: main.py [-h] [--optional OPTIONAL] required    

The description of the project

positional arguments:
  required             The required argument

options:
  -h, --help           show this help message and exit
  --optional OPTIONAL  The optional argument

by @maseoff
```

**Пояснение.** Модуль `argparse` полагает, что аргумент является опциональным, если его имя начинается с `-`.

**Пример.** Взаимодействие с аргументами.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument("required", help="The required argument")
parser.add_argument("--optional", help="The optional argument")

args = parser.parse_args()

print("required:", args.required)
print("optional:", args.optional)

**Запуск.** Запустите в терминале следующую команду: `python ./main.py 42`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
required: 42
optional: None
```

**Запуск.** Запустите в терминале следующую команду: `python ./main.py 42 --optional forty-two`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
required: 42
optional: forty-two
```

**Замечание.** Если опциональный аргумент не был передан, то его значение по умолчанию - `None`.

**Рассуждение.** Использование параметра `--optional` предполагает наличие аргумента, который, обратите внимание, помечен в подсказке как `OPTIONAL` - используется капитализация имени параметра. Допустим, нам не нравится это имя - давайте его поменяем.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument("--limit", help="Memory limit", metavar="KiB")

parser.parse_args()

**Запуск.** Запустите в терминале следующую команду: `python ./main.py -h`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
usage: main.py [-h] [--limit KiB]

The description of the project

options:
  -h, --help   show this help message and exit
  --limit KiB  Memory limit

by @maseoff
```

**Упражнение.** Убедитесь в том, что при отсутствии параметра `metavar` вместо `KiB` будет использовано имя `LIMIT`.

**Пример.** Задать несколько имен для одного параметра.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument("-lim", "--limit", help="Memory limit", metavar="KiB")

parser.parse_args()

**Запуск.** Запустите в терминале следующую команду: `python ./main.py -h`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
usage: main.py [-h] [-lim KiB]

The description of the project

options:
  -h, --help            show this help message and exit
  -lim KiB, --limit KiB
                        Memory limit

by @maseoff
```

**Пример.** Задать значение по умолчанию для некоторго аргумента.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument(
    "-lim",
    "--limit",
    metavar="KiB",
    default=1024,
    type=int,
    help="Memory limit",
)

args = parser.parse_args()
print("limit:", args.limit)

**Запуск.** Запустите в терминале следующую команду: `python ./main.py --limit 512`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
limit: 512
```

**Запуск.** Запустите в терминале следующую команду: `python ./main.py`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
limit: 1024
```

**Рассуждение.** Одним из наиболее частых применений опциональных параметров является выставление флагов. Например, Вы могли видеть как в приложениях выставляют флаг `-f` или `--force`. Модуль `argparse` позволяет задать аналогичное поведение.

**Пример.** Создание флага для консольного приложения.

In [ ]:
# %file: ./main.py

import argparse

parser = argparse.ArgumentParser(
    description="The description of the project",
    epilog="by @maseoff",
)

parser.add_argument(
    "-f",
    "--force",
    default=False,
    action="store_true",
)

args = parser.parse_args()
print("force:", args.force)

**Запуск.** Запустите в терминале следующую команду: `python ./main.py`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
force: False
```

**Запуск.** Запустите в терминале любую из следующих команд: `python ./main.py -f` или `python ./main.py --force`

**Ожидаемый вывод.** Если все прошло так, как нужно, то Вы увидите сообщение:

```
force: True
```

**Замечание.** Параметр `action` определяет, какую функцию несет в себе тот или иной параметр. По умолчанию `action` равен `"store"`. Помимо `"store_true"`, есть и `"store_false"`.

**Спойлер.** На семинаре:

- Пример создания консольной утилиты.